In [ ]:
# default_exp utility.markdown

# utility.markdown

> Markdown processing and displaying

## Embed markdown file

In [ ]:
#export
from IPython.display import display, Markdown
from mathbook.utility.string import replace_string_by_indices
import os.path
from pathlib import Path
import re
import urllib.parse

### Links

I often embed markdown files here via ``display(Markdown(text))`` (see ``embed_markdown_file``). This is useful when I want to state the same thing multiple times (e.g. definitions) because I can write the statement in one location and it will automatically be updated whenever I make changes to the statement (e.g. fixing errata).

One obstruction to this is that links in the markdown files cannot be expected to link to where I want them to - if I were to make links to (local) notebook files during editing, then the links will not link to webpages when I upload the website.

The [``embed_markdown_file``](#embed_markdown_file) method circumvents this obstruction - it allows me to choose whether the links will lead to (local) notebooks or the website. I will format links in the markdown files without specifying the precise locations, e.g. ``[limit](limit_of_a_function_at_a_point)`` will show the text ``limit`` and link to either a (local) notebook ``limit_of_a_function_at_a_point.ipynb`` or a webpage ``limit_of_a_function_at_a_point.html`` in the website.

>**Note** The notebook files and the corresponding html files that are generated during ``nbdev_build_docs`` need to have the same name, e.g. ``function.ipynb`` needs to generate ``function.html``.

In [ ]:
#export
def find_links_in_markdown_text(text):
    """Returns ranges in the markdown text string where links occur.
    """
    matches = re.finditer('\\[.*\\]\\(.*\\)', text)
    return [match.span() for match in matches]

**Parameters**
- ``text`` - str

**Returns**
- list of tuples. Each tuple is of the form ``(a,b)`` where ``text[a:b]`` is a markdown link.

**Examples**

In [ ]:
sample_text = '123hi [thing](link) blah'
sample_range = find_links_in_markdown_text(sample_text)
assert len(sample_range) == 1
assert sample_text[sample_range[0][0]:sample_range[0][1]] == '[thing](link)'

In [ ]:
#export
#TODO make it so that an anchor of the form '#anchor' is anchored to the current document.
def link_with_anchor(link_name, location='website'):
    """Formats link so that the file extension comes before
    the pound sign of the anchor.
    """
    assert link_name and location in ['website', 'notebook']
    link_parts = link_name.split('#')
    assert len(link_parts) <= 2
    anchor = '' if len(link_parts) == 1 else f'#{link_parts[1]}'
    extension = '.html' if location == 'website' else '.ipynb'
    return f'{link_parts[0]}{extension}{anchor}' 

**Parameters**
- link_name - str
    - Should not be empty. Should be of the form `'link'` or `'link#anchor'`, i.e. should not have surrounding parentheses. Does not have a file extension, e.g. `'.html'` or `'.ipynb'`.
- location - `'website'` or `'notebook'`.

**Returns**
- The same as ``link_name``, except with the appropriate file extension added in before the anchor, if any and with the parentheses removed.

**Examples**

In [ ]:
sample_link_name = 're#re.split'
sample_output = link_with_anchor(sample_link_name, location ='website')
assert sample_output == 're.html#re.split'
sample_output = link_with_anchor(sample_link_name, location ='notebook')
assert sample_output == 're.ipynb#re.split'

sample_link_name = 'hi#'
sample_output = link_with_anchor(sample_link_name, location ='website')
assert sample_output == 'hi.html#'

In [ ]:
#export
WEBSITE_URL = 'https://hyunjongkimmath.github.io/mathbook/'
def get_formatted_markdown_link(unformatted_link, location='website'):
    """Formats markdown styled link string.
    """
    assert location in ['website', 'notebook']
    assert re.fullmatch(pattern='\\[.*\\]\\(.*\\)', string=unformatted_link)
    matches = re.finditer('\\(.*\\)', unformatted_link)
    start, end = next(matches).span()
    link_name = unformatted_link[start+1:end-1]
    relative_link = link_with_anchor(link_name, location=location)
    if location == 'website':
        link = f'({WEBSITE_URL}{relative_link})'
    else:
        link = f'({relative_link})'
    return replace_string_by_indices(unformatted_link, (start, end),
                                     replace_with = link)

**Parameters**
- ``unformatted_link`` - str
    - Of the form ``[text](link)``.
- ``location`` - ``'website'`` or ``'notebook'``.

**Returns**
- str, of the form ``[text](link.html)`` or ``[text](link.ipynb)`` depending
on ``location``.

**Examples**

In [ ]:
sample_link = '[function](function)'
sample_formatted_link = get_formatted_markdown_link(sample_link, location='website')
assert sample_formatted_link == '[function](https://hyunjongkimmath.github.io/mathbook/function.html)'
sample_formatted_link = get_formatted_markdown_link(sample_link, location='notebook')
assert sample_formatted_link == '[function](function.ipynb)'

sample_link = '[limit](limit#limit_of_a_function_at_a_point)'
sample_formatted_link = get_formatted_markdown_link(sample_link, location='website')
assert sample_formatted_link == '[limit](https://hyunjongkimmath.github.io/mathbook/limit.html#limit_of_a_function_at_a_point)'
sample_formatted_link = get_formatted_markdown_link(sample_link, location='notebook')
assert sample_formatted_link == '[limit](limit.ipynb#limit_of_a_function_at_a_point)'

In [ ]:
#export
# TODO make it so that if a link is already specified with a format, then the format
# is preserved.
def replace_all_links_with_formatted_links(text, location='website'):
    """Returns a modification of ``text`` with all links formatted."""
    assert location in ['website', 'notebook']
    link_ranges = find_links_in_markdown_text(text)
    formatted_links = [get_formatted_markdown_link(text[start:end], location=location)
                          for start, end in link_ranges]
    return replace_string_by_indices(text, link_ranges, formatted_links)
    

**Parameters**
- ``text`` - str
- ``location`` - `'website'` or `'notebook'`.

### Footnotes

I usually use footnotes in markdowns to make citations or to indicate notations. An example of the latter kind of footnotes is "$\operatorname{Pic}(X)$ denotes the Picard group of the variety $X$", where the term "Picard group" is linked to a page containing a definition of the term.

Although this can be time consuming for the writer to put together, the reader does not have to spend a great effort to figure out where the notation is introduced. This is particularly nice especially there is much notation exclusive to the writer's field or niche.

The writer also can quickly remind themselves of the notation whenever necessary. This is particularly useful when the writer is learning something new for the first time.

In markdown for jupyter notebook, however, the footnotes seem to be links by default. This thus has the same problem as that in [the section above](#Links). In other words, I also have to make sure that footnotes link to the correct location.

In [ ]:
#export
def find_footnotes_in_markdown_text(text):
    """Returns ranges in the markdown text string where footnotes occur.
    """
    matches = re.finditer('\\[\\^.*\\]\\:.*', text)
    return [match.span() for match in matches]

**Parameters**
- ``text`` - str

**Returns**
- list of tuples. Each tuple is of the form ``(a,b)`` where ``text[a:b]`` is a markdown footnote.

**Examples**

In [ ]:
sample_text = 'some stuff[^1][^2]\n[^1]:link#do\n[^2]:other_link'
sample_ranges = find_footnotes_in_markdown_text(sample_text)
assert len(sample_ranges) == 2
a, b = sample_ranges[0]
c, d = sample_ranges[1]
assert sample_text[a:b] == "[^1]:link#do"
assert sample_text[c:d] == "[^2]:other_link"

In [ ]:
#export
def replace_all_footnotes_with_formatted_footnotes(text, location='website'):
    """Returns a modification of ``text`` with all footnote links formatted."""
    assert location in ['website', 'notebook']
    footnote_ranges = find_footnotes_in_markdown_text(text)
    formatted_footnotes = [link_with_anchor(text[start:end], location=location)
                              for start, end in footnote_ranges]
    return replace_string_by_indices(text, footnote_ranges, formatted_footnotes)

**Parameters**
- ``text`` - str
- ``location`` - `'website'` or `'notebook'`.

**Examples**

In [ ]:
sample_text = 'some stuff[^1][^2]\n[^1]:link#do\n[^2]:other_link'
replace_all_footnotes_with_formatted_footnotes(sample_text, location='website')

'some stuff[^1][^2]\n[^1]:link.html#do\n[^2]:other_link.html'

### Final method

In [ ]:
#export
def embed_markdown_file(file_path, display_file_link=False, location='website'):
    """Embed markdown file content into ipynb.
    """
    with open(file_path, 'r') as markdown_file:
        text = markdown_file.read()
        text = replace_all_links_with_formatted_links(text, location=location)
        text = replace_all_footnotes_with_formatted_footnotes(text, location=location)
        display(Markdown(text))
        markdown_file.close()

**Parameters**
- ``file_path`` - Path-like
- ``display_file_link`` - bool
    - If True, then displays a link to an edit mode ipynb for 
    the markdown file at ``file_path``. This should only be 
    set to True inside an ipynb environment such as jupyter.

**Examples**

In [ ]:
embed_markdown_file('definition.limit_of_a_function_at_a_finite_input.md', location='website')

Let $f: D \to \mathbb{R}$ be a [real valued function](definition.real_valued_function.ipynb#Real-valued-function), and assume that $f$ is defined near $a$.

We write $$\lim\limits_{x \to a} f(x) = c$$ if for every $\epsilon > 0$ there is a $\delta > 0$ such that whenever $|x-a| < \delta$ for $x \in D$[^1] 

[^1]:notation.basic.ipynb#$\in$